# Analysis of NetLogo PVI (pedestrian-vehicle interaction) model

This notebook does:
- Reads netlogo data (output) `log.csv`
- Clean data (we erase the first agents in the simulation, since very low number of initial pedestrian in the intersection skews the model. We then can set in the large-scale model a threshold of intersection crowdness based on number of agents and geometry of intersection. This is a topic for further research. For now, 50 pedestrians enter the simulation every 30 sec.)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.matplotlib.style.use("seaborn")

# Read data from NetLogo

In [ ]:
dfl = pd.read_csv('../netlogo/utility/tests/log.csv',names=["id","start","end","vmax","num-cars","num-peds","num-cross","in","out","time"])
dfl.shape

Lets try to earase first peds with `num-peds = 49`.

In [ ]:
dfl.drop(dfl[dfl["num-peds"]==49].index,inplace=True)

In [ ]:
#164253 ticks /120 fps ~ 22.8 min
data = dfl.drop(['id','start','end','vmax','num-cross','in','out'],axis=1) 
data['time']= data['time']/120
# data['vmax'] = data['vmax'].mean() * 0.5 * 120
data.to_csv('./netlogo_output.csv')
data.head()

In [ ]:
plt.plot(dfl["time"],dfl["vmax"],'o')
plt.xlabel("ticks")
plt.ylabel("vmax");

In [ ]:
ax = plt.axes(projection='3d')
import numpy as np
x = dfl["num-cars"]
y = dfl["num-peds"]
z = dfl["time"]
ax.scatter(x,y,z)
ax.set_xlabel("cars")
ax.set_ylabel("peds")
ax.set_zlabel("time")

In [ ]:
import seaborn as sns
sns.pairplot(dfl)

# Multiple Linear Regression

Based on [Web](https://www.analyticsvidhya.com/blog/2021/05/multiple-linear-regression-using-python-and-scikit-learn/)

In [ ]:
#separate attributes from predicting attribute
x = dfl.drop(['id','start','end','in','out','time'],axis=1) 
#separate the predicting attribute into Y for model training
y = dfl['time']

In [ ]:
#import train_test_split from sklearn
from sklearn.model_selection import train_test_split
#split the data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)

In [ ]:
#import module
from sklearn.linear_model import LinearRegression
#create an objeect of LinearRegression class
LR = LinearRegression()
#fitting the training data
LR.fit(x_train,y_train)

In [ ]:
y_prediction = LR.predict(x_test)

In [ ]:
#import r2_score module
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
#predicting the accuracy score
score=r2_score(y_test,y_prediction)
meansqerr = mean_squared_error(y_test,y_prediction)
print(f'r2_score is: {score}')
print(f'mean sqrd error is:{meansqerr}')
print(f'root mean sqr err is:{np.sqrt(meansqerr)}')

# Random Forest

In [ ]:
#separate attributes from predicting attribute
x = dfl.drop(['id','start','end','in','out','time'],axis=1) 
#separate the predicting attribute into Y for model training
y = dfl['time']

In [ ]:
#import train_test_split from sklearn
from sklearn.model_selection import train_test_split
#split the data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor()

#train the model
clf.fit(x_train,y_train)
#predict the model
y_prediction = clf.predict(x_test)

In [ ]:
#import r2_score module
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
#predicting the accuracy score
score=r2_score(y_test,y_prediction)
meansqerr = mean_squared_error(y_test,y_prediction)
print(f'r2_score is: {score}')
print(f'mean sqrd error is:{meansqerr}')
print(f'root mean sqr err is:{np.sqrt(meansqerr)}')

# Gaussian Process Regression (not working)

In [ ]:
# #separate attributes from predicting attribute
# x = dfl.drop(['id','start','end','in','out','time'],axis=1) 
# #separate the predicting attribute into Y for model training
# y = dfl['time']

In [ ]:
# #import train_test_split from sklearn
# from sklearn.model_selection import train_test_split
# #split the data
# x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)

In [ ]:
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
# from sklearn import preprocessing

# #create a GP model
# kernel = C(1.0,(1e-3,1e3))*RBF(1,(1e-2,1e2))
# gp = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=100,)

# #scale the data
# scaler = preprocessing.StandardScaler().fit(x_train)
# x_train_scaled = scaler.transform(x_train)

# #fit the data
# gp.fit(x_train_scaled,y_train)

# #predict
# y_pred, sigma = gp.predict(x_test, return_std=True)

In [ ]:
# #import r2_score module
# from sklearn.metrics import r2_score
# from sklearn.metrics import mean_squared_error
# #score
# gp.score(x_train_scaled,y_train)
# #predicting the accuracy score
# score=r2_score(y_test,y_pred)
# meansqerr = mean_squared_error(y_test,y_pred)
# print(f'r2_score is: {score}')
# print(f'mean sqrd error is:{meansqerr}')
# print(f'root mean sqr err is:{np.sqrt(meansqerr)}')

# Krigging (KPLS) with SMT library

In [ ]:
dfl.shape

In [ ]:
#separate attributes from predicting attribute
# x = dfl.drop(['id','start','end','vmax','in','out'],axis=1) 
x = dfl.drop(['id','start','end','in','out'],axis=1) 
xg = x.groupby(by=['num-cars','num-peds','num-cross']).mean()
print(xg.shape)


In [ ]:
xg.head()

In [ ]:
# need to use to_records since xg is a group df
x = xg.to_records(index=True)
x = pd.DataFrame(x).to_numpy()
y = x[:,-1]
x = x[:,:-1]
print(x.shape,y.shape)

In [ ]:
#import train_test_split from sklearn
from sklearn.model_selection import train_test_split
#split the data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from smt.surrogate_models import KPLS

sm = KPLS(theta0=[1e-2])
sm.set_training_values(x_train,y_train)
sm.train()

y_prediction = sm.predict_values(x_test)

fig, axs = plt.subplots(1)
axs.plot(y_test, y_prediction,'r.')
axs.plot(y_test,y_test,'k-')

In [ ]:
#import r2_score module
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
#predicting the accuracy score
score=r2_score(y_test,y_prediction)
meansqerr = mean_squared_error(y_test,y_prediction)
print(f'r2_score is: {score}')
print(f'mean sqrd error is:{meansqerr}')
print(f'root mean sqr err is:{np.sqrt(meansqerr)}')

# Data from paper

In [ ]:
df = pd.read_csv('./data.csv')
df.head()

In [ ]:
def plot_data(num=1):
    fig, ax = plt.subplots(figsize=(2,10))
    for num in range(5):
        x = df[df['id']==num].x_obs
        y = df[df['id']==num].y_obs
        xe = df[df['id']==num].x_est
        ye = df[df['id']==num].y_est
        ax.plot(x,y,label=f'obs{num}')
        ax.plot(xe,ye,label=f'est{num}')
    plt.legend(loc='lower left',bbox_to_anchor=(1.04, 0),fancybox=True,shadow=True)

In [ ]:
plot_data(5)